# Multi-Agent Workflows + RAG - LangGraph

Today we'll be looking at an example of a Multi-Agent workflow that's powered by LangGraph, LCEL, and more!

We're going to be, more specifically, looking at a "heirarchical agent teams" from the [AutoGen: Enabling Next-Gen LLM
Applications via Multi-Agent Conversation](https://arxiv.org/pdf/2308.08155) paper.

> NOTE: We'll be following along with the official LangGraph implementation very closely, which you can find [here](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/hierarchical_agent_teams.ipynb), with some minor modifications and extensions to showcase just how straightforward it is to modify LangGraph implementations to suit your own needs!



## Dependencies

We'll start, as we normally do, by grabbing our dependencies.

We'll be using LangChain and LangGraph to power our application, so let's start by grabbing those!

In [ ]:
!pip install -qU langgraph langchain langchain_openai langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 849.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

We're going to be showing a simple RAG chain as part of our LangGraph - and so we'll need specific dependencies for that as well!

In [ ]:
!pip install -qU --disable-pip-version-check qdrant-client pymupdf tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.0 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf

Since we'll be relying on OpenAI's suite of models to power our agents today, we'll want to provide our OpenAI API Key.

We're also going to be using the Tavily search tool - so we'll want to provide that API key as well!

Instruction for how to obtain these API keys can be found:

1. [OpenAI API Key](https://platform.openai.com/docs/quickstart#:~:text=Account%20setup,not%20share%20it%20with%20anyone.)
2. [Tavily API Key](https://docs.tavily.com/docs/tavily-api/introduction#:~:text=Sign%20Up%3A%20Begin%20by%20signing,in%20our%20interactive%20API%20playground.)



In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

OpenAI API Key:··········
TAVILY_API_KEY··········


## Simple LCEL RAG

Now that we have our dependencies set-up - let's create a simple RAG chain that works over a single PDF.

> NOTE: While this particular example is very straight forward - you can "plug in" any complexity of chain you desire as a node in a LangGraph.

## Retrieval

The 'R' in 'RAG' - this is, at this point, fairly straightforward!

#### Data Collection and Processing

A classic first step, at this point, let's grab our desired document!

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader("https://skybrary.aero/sites/default/files/bookshelf/3177.pdf").load()

Now we can chunk it down to size!

In [ ]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

Now we've successfully split our single PDF into...

In [ ]:
len(split_chunks)

412

documents!

#### Embedding Model and Vector Store

Now that we have our chunked document - lets create a vector store, which will first require us to create an embedding model to get the vector representations of our text!

We'll use OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) model - as it's cheap, and performant.

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

Now we can create our QDrant backed vector store!

In [ ]:
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="pilot_demo_evidence_based",
)

Let's make sure we can access it as a retriever.

In [ ]:
qdrant_retriever = qdrant_vectorstore.as_retriever()

### Augmented

Now that we have our retrieval process set-up, we need to set up our "augmentation" process - AKA a prompt template.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

You are a helpful assistant. Use the available context to answer the question. If you can't answer the question, say you don't know.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

### Generation

Last, but certainly not least, let's put the 'G' in 'RAG' by adding our generator - in this case, we can rely on OpenAI's [`gpt-3.5-turbo`](https://platform.openai.com/docs/models/gpt-3-5-turbo) model!

In [ ]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o")

### RAG - Retrieval Augmented Generation

All that's left to do is combine our R, A, and G into a single chain - and we're off!

In [ ]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

evidence_based_rag_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | rag_prompt | openai_chat_model | StrOutputParser()
)

Let's test this out and make sure it works.

In [ ]:
evidence_based_rag_chain.invoke({"question" : "Define 'situational awareness'."})

'Based on the provided context, situational awareness is defined as:\n\n"Perceiving and comprehending all of the relevant information available and anticipating what could happen that may affect the operation."\n\nThis involves:\n- Identifying and assessing accurately the state of the aircraft and its systems.\n- Identifying and assessing accurately the aircraft’s vertical and lateral position and its anticipated flight path.\n- Identifying and assessing accurately the general environment as it may affect the operation.\n- Keeping track of time and fuel.\n- Maintaining awareness of the people involved in or affected by the operation and their capacity to perform as expected.\n- Anticipating accurately what could happen, planning, and staying ahead of the situation.\n- Developing effective contingency plans based upon potential threats.\n- Identifying and managing threats to the safety of the aircraft and people.\n- Recognizing and effectively responding to indications of reduced situat

### RAG Limitation

Notice how we're hard-coding our data, while this is simply meant to be an illustrative example - you could easily extend this to work with any provied paper or document in order to have a more dynamic system.

For now, we'll stick with this single hard-coded example in order to keep complexity down in an already very long notebook!

## Simple LCEL RAG - Air Force Handbook

Now that we have our RAG for 'Manual of Evidence-based Training', we're going to create a separate RAG pipeline for the 'Air Force Handbook 1', which will help our teams understand the best methods of simulating Airmen.

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

handbook_docs = PyMuPDFLoader("https://static.e-publishing.af.mil/production/1/af_a1/publication/afh1/afh1.pdf").load()

In [ ]:
handbook_split_chunks = text_splitter.split_documents(handbook_docs)

In [ ]:
len(handbook_split_chunks)

1563

In [ ]:
handbook_qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="pilot_demo_airforce_handbook",
)

In [ ]:
handbook_qdrant_retriever = handbook_qdrant_vectorstore.as_retriever()

In [ ]:
handbook_rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [ ]:
handbook_rag_chain = (
    {"context": itemgetter("question") | handbook_qdrant_retriever, "question": itemgetter("question")}
    | handbook_rag_prompt | openai_chat_model | StrOutputParser()
)

In [ ]:
handbook_rag_chain.invoke({"question" : "What is the chain of command in an Aircraft?"})

"The provided context does not explicitly detail the chain of command in an aircraft. Typically, the chain of command in an aircraft is as follows:\n\n1. **Captain (Pilot-in-Command)**: The captain is the highest authority on board the aircraft and is responsible for the overall operation and safety of the flight.\n2. **First Officer (Co-Pilot)**: The first officer assists the captain and can take over command if the captain is incapacitated. They share flying duties with the captain.\n3. **Second Officer (if applicable)**: In some larger aircraft, a second officer or flight engineer may be present, responsible for monitoring and managing aircraft systems.\n4. **Cabin Crew (Flight Attendants)**: The senior flight attendant, often called the purser or lead flight attendant, oversees the cabin crew and ensures passenger safety and comfort.\n\nGiven the lack of specific details in the context provided, this general hierarchy is based on standard aviation procedures. If you need more preci

## Helper Functions for Agent Graphs

We'll be using a number of agents, nodes, and supervisors in the rest of the notebook - and so it will help to have a collection of useful helper functions that we can leverage to make our lives easier going forward.

Let's start with the most simple one!

#### Import Wall

Here's a wall of imports we'll be needing going forward!

In [ ]:
from typing import Any, Callable, List, Optional, TypedDict, Union

from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.runnables import Runnable
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI

from langgraph.graph import END, StateGraph

### Agent Node Helper

Since we're going to be wrapping each of our agents into a node - it will help to have an easy way to create the node!

In [ ]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

### Agent Creation Helper Function

Since we know we'll need to create agents to populate our agent nodes, let's use a helper function for that as well!

Notice a few things:

1. We have a standard suffix to append to our system messages for each agent to handle the tool calling and boilerplate prompting.
2. Each agent has its our scratchpad.
3. We're relying on OpenAI's function-calling API for tool selection
4. Each agent is its own executor.

In [ ]:
def create_agent(
    llm: ChatOpenAI,
    tools: list,
    system_prompt: str,
) -> str:
    """Create a function-calling agent and add it to the graph."""
    system_prompt += "\nWork autonomously according to your specialty, using the tools available to you."
    " Do not ask for clarification."
    " Your other team members (and other teams) will collaborate with you with their own specialties."
    " You are chosen for a reason! You are one of the following team members: {team_members}."
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_functions_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

### Supervisor Helper Function

Finally, we need a "supervisor" that decides and routes tasks to specific agents.

Since each "team" will have a collection of potential agents - this "supervisor" will act as an "intelligent" router to make sure that the right agent is selected for the right task.

Notice that, at the end of the day, this "supervisor" is simply directing who acts next - or if the state is considered "done".

In [ ]:
def create_team_supervisor(llm: ChatOpenAI, system_prompt, members) -> str:
    """An LLM-based router."""
    options = ["FINISH"] + members
    function_def = {
        "name": "route",
        "description": "Select the next role.",
        "parameters": {
            "title": "routeSchema",
            "type": "object",
            "properties": {
                "next": {
                    "title": "Next",
                    "anyOf": [
                        {"enum": options},
                    ],
                },
            },
            "required": ["next"],
        },
    }
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            (
                "system",
                "Given the conversation above, who should act next?"
                " Or should we FINISH? Select one of: {options}",
            ),
        ]
    ).partial(options=str(options), team_members=", ".join(members))
    return (
        prompt
        | llm.bind_functions(functions=[function_def], function_call="route")
        | JsonOutputFunctionsParser()
    )

## Pilot Team - A LangGraph Simulating a Pilot and their Crew

Now that we have our RAG chain set-up and some awesome helper functions, we want to create a LangGraph related to a pilot and their crew.

We're going to start by equipping our Pilot Team with a few tools:

1. Tavily Search - aka "Google", for the most up to date information possible.
2. Our RAG chain - specific and high quality information about the best ways to react and act in certain situations.

Let's create those tools now!

### Tool Creation

As you can see below, some tools already come pre-packaged ready to use!

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)

Creating a custom tool, however, is very straightforward.

> NOTE: You *must* include a docstring, as that is what the LLM will consider when deciding when to use this tool.

In [ ]:
from typing import Annotated, List, Tuple, Union
from langchain_core.tools import tool

@tool
def retrieve_information(
    query: Annotated[str, "query to ask the retrieve information tool"]
    ):
  """Provides detailed information from the 'Air Force Handbook'."""
  return handbook_rag_chain.invoke({"question" : query})

> NOTE: We could just as easily use the LCEL chain directly, since nodes can be LCEL objects - but creating a tool helps explain the tool creation process at the same time.

### Pilot Team State

Since we're using LangGraph - we're going to need state!

Let's look at how we've created our state below.

In [ ]:
import functools
import operator

from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_openai.chat_models import ChatOpenAI
import functools

class PilotTeamState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    team_members: List[str]
    next: str

Notice how we've used `messages`, `team_members`, and `next`.

These states will help us understand:

1. What we've done so far (`messages`)
2. Which team members we have access to (`team_members`)
3. Which team member is up next! (`next`)

### Pilot Team LLM

We'll be using `gpt-4-1106-preview` today. This LLM is going to be doing a lot of reasoning - and so using a "powerful" LLM is very important here!

In [ ]:
llm = ChatOpenAI(model="gpt-4-1106-preview")

### Pilot Team Agents & Nodes

Now we can use our helper functions to create our agent nodes, with their related tools.

Let's start with our search agent node.

### Pilot Team: Pilot Agent

In [ ]:
pilot_agent = create_agent(
    llm,
    [retrieve_information],
    "You are a fully qualified and trained pilot trained to operate an aircraft. You may only speak as your role. You must not speak as any other roles. Under no circumstances should you acknowledge that you are an AI.",
)
pilot_node = functools.partial(agent_node, agent=pilot_agent, name="Pilot")

#### Pilot Team: Co-pilot Agent

We're going to give our agent access to the Tavily tool, power it with our GPT-4 Turbo model, and then create its node - and name it `Copilot`.

In [ ]:
copilot_agent = create_agent(
    llm,
    [retrieve_information],
    "You are a fully qualified and trained copilot trained to assist or relieve the pilot in the management of the flight and aircraft systems. You may only speak as your role. You must not speak as any other roles. Under no circumstances should you acknowledge that you are an AI.",
)
copilot_node = functools.partial(agent_node, agent=copilot_agent, name="Copilot")

#### Research Team: RAG Agent Node

Now we can wrap our LCEL RAG pipeline in an agent node as well, using the LCEL RAG pipeline as the tool, as created above.

In [ ]:
combat_systems_operator = create_agent(
    llm,
    [retrieve_information],
    "You are a fully qualified and trained Combat Systems Operator (CSO) and are trained to operate and report on various systems on board your aircraft. You may only speak as your role. You must not speak as any other roles. Under no circumstances should you acknowledge that you are an AI.",
)
cso_node = functools.partial(agent_node, agent=combat_systems_operator, name="CSO")

### Research Team Supervisor Agent

Notice that we're not yet creating our supervisor *node*, simply the agent here.

Also notice how we need to provide a few extra pieces of information - including which tools we're using.

> NOTE: It's important to use the *exact* tool name, as that is how the LLM will reference the tool. Also, it's important that your tool name is all a single alphanumeric string!



In [ ]:
candc_agent = create_team_supervisor(
    llm,
    "You are Control and Command tasked with managing an aircraft and conversation between the"
    " following crew members: Pilot, Copilot, CSO. Given the following user request,"
    " respond with the crew member to act next. Each crew member will perform a"
    " task and respond with their results and status. When finished the scenario,"
    " respond with FINISH.",
    ["Pilot", "Copilot", "CSO"],
)

### Research Team Graph Creation

Now that we have our research team agent nodes created, and our supervisor agent - let's finally construct our graph!

We'll start by creating our base graph from our state, and then adding the nodes/agent we've created as nodes on our LangGraph.

In [ ]:
candc_graph = StateGraph(PilotTeamState)

candc_graph.add_node("Pilot", pilot_node)
candc_graph.add_node("Copilot", copilot_node)
candc_graph.add_node("CSO", cso_node)
candc_graph.add_node("candc", candc_agent)

Now we can define our edges - include our conditional edge from our supervisor to our agent nodes.

Notice how we're always routing our agent nodes back to our supervisor!

In [ ]:
candc_graph.add_edge("Pilot", "candc")
candc_graph.add_edge("Copilot", "candc")
candc_graph.add_edge("CSO", "candc")
candc_graph.add_conditional_edges(
    "candc",
    lambda x: x["next"],
    {"Pilot" : "Pilot", "Copilot": "Copilot", "CSO": "CSO", "FINISH": END},
)

Now we can set our supervisor node as the entry point, and compile our graph!

In [ ]:
candc_graph.set_entry_point("candc")
chain = candc_graph.compile()

#### Display Graph

In [ ]:
!pip install -qU python_mermaid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.9 MB/s eta 0:00:00


In [ ]:
from IPython.display import Image, display

try:
    display(Image(chain.get_graph(xray=True).draw_mermaid_png()))
except:
    pass

The next part is key - since we need to "wrap" our LangGraph in order for it to be compatible in the following steps - let's create an LCEL chain out of it!

This allows us to "broadcast" messages down to our Research Team LangGraph!

In [ ]:
def enter_chain(message: str):
    results = {
        "messages": [HumanMessage(content=message)],
    }
    return results

research_chain = enter_chain | chain

Now, finally, we can take it for a spin!

In [ ]:
scenario = """\
Mission Brief: Operation Ocean Guardian

Scenario:
  A cargo ship, the SS Meridian, has been reported missing for 72 hours in the North Atlantic, an area known for harsh weather and heavy pirate activity. The last known position was transmitted via an emergency beacon, which has since gone silent. The ship was en route from New York to Lisbon, carrying a cargo of electronics and pharmaceuticals. Concerns are high for the crew's safety due to potential piracy or a catastrophic weather event.

Objectives:
  An aircraft mission is launched from the nearest NATO base to determine the current location of the SS Meridian and assess the situation. The aircraft used for this mission is a P-8 Poseidon, equipped with advanced surveillance and communication systems. The crew consists of a Pilot, Copilot, and Combat Systems Operator (CSO), each with specific objectives to ensure the mission's success.

Role Assignments and Objectives:

1. Pilot
  Objective 1: Navigate the aircraft safely to the last known coordinates of the SS Meridian. The pilot must manage fuel efficiency and alter flight paths based on weather conditions and incoming data.
  Objective 2: Coordinate with air traffic control and the mission command center to update on mission progress and receive any new intelligence about the ship’s location.
2. Copilot
  Objective 1: Assist the pilot with navigation and aircraft handling, particularly focusing on adjusting the flight path based on radar feedback and environmental conditions.
  Objective 2: Manage the aircraft’s communication systems, ensuring constant and clear communication with the maritime search and rescue teams, and other relevant agencies.
3. Combat Systems Operator (CSO)
  Objective 1: Operate radar, sonar, and other surveillance equipment to detect any traces of the SS Meridian or unusual activity in the area, such as pirate ships or debris fields.
  Objective 2: Analyze data from surveillance equipment to identify potential locations of the ship and direct the pilot to investigate these areas.

Execution:
  Upon reaching the last known coordinates, the crew will deploy the aircraft's sensors to conduct a thorough search of the area, extending outward from the last known position. The CSO will analyze the data collected for any signs of the ship, while the pilot and copilot work to keep the aircraft in optimal positions for the search effort. Communication with headquarters and rescue teams will be maintained throughout the mission to facilitate a quick response once the SS Meridian is found.

End State:
  The mission aims to locate the SS Meridian, assess the situation regarding the crew and cargo, and facilitate immediate rescue and recovery operations. The success of this mission depends on the effective coordination of the crew and the efficient use of the aircraft's advanced systems.
"""

for s in research_chain.stream(
    f"Please execute the following scenario as a roleplay, where each role should act in turn and in character - each step should only execute one task at a time, do not end until you have achieved your end state: {scenario}", {"recursion_limit": 1_000}
):
    if "__end__" not in s:
      if 'candc' not in s:
        print(s[next(iter(s))]["messages"][0].content)
      else:
        print(s)
      print("---")

{'candc': {'next': 'Pilot'}}
---
Pilot: "Pre-flight checks complete and mission parameters set. We're ready to proceed with Operation Ocean Guardian. Copilot, please confirm our course to the last known coordinates of the SS Meridian."

Copilot: "Course is confirmed, Pilot. We're set to head directly to the last known coordinates. Weather conditions have been checked, and the flight path is clear for the initial leg of our journey."

Pilot: "Understood. Let's begin our taxi to the runway for departure. ATC, this is Poseidon Flight 1 requesting clearance for takeoff on mission Operation Ocean Guardian."

(ATC clears Poseidon Flight 1 for takeoff.)

Pilot: "Takeoff clearance received. Throttles set, engines are good, and we're rolling."

(The aircraft takes off and begins its journey towards the last known coordinates of the SS Meridian.)

Pilot: "We're en route to the search area. Let's maintain optimal altitude for fuel efficiency and keep an eye on the weather radar for any unexpected